In [1]:
import os

import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.colors import Normalize

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

In [2]:
from tqdm.auto import tqdm

## Data for GEO

In [3]:
name2map = {
    'Neimenggu':'Nei Mongol',
    'Ningxia':'Ningxia Hui',
    'Xinjiang':'Xinjiang Uygur',
    'Macau':'Macao',
}

geo_data = {}

for record in shpreader.Reader('./data_GADM/gadm36_CHN_1.shp').records():
    name = record.attributes['NAME_1']
    geo  = record.geometry
    geo_data[name] = geo

for sp in ['HKG','MAC','TWN']:
    record = list(shpreader.Reader('./data_GADM/gadm36_{0}_0.shp'.format(sp)).records())[0]
    name = record.attributes['NAME_0']
    geo = record.geometry
    geo_data[name] = geo

date_str = pd.date_range(start='2020-01-22',end='2020-03-23').strftime('%Y-%m-%d')

## Load data from DXY
- Keep the max value for a day
- Define `existed` = `confirmed` - `cured` - `dead`

In [4]:
china_df = (
    pd.read_csv('./DXYArea.csv')
    .query('countryEnglishName=="China"')
    .query('province_confirmedCount!=0')
    .loc[:,['provinceEnglishName','province_confirmedCount','province_curedCount','province_deadCount','updateTime']]
    .rename(columns={'provinceEnglishName':'Name','province_confirmedCount':'confirmed','province_curedCount':'cured','province_deadCount':'dead'})
    .copy()
)

china_df['date_str'] = china_df['updateTime'].apply(lambda x:pd.to_datetime(x).date().strftime('%Y-%m-%d'))

china_df = (
    china_df
    .groupby(['Name','date_str'],sort=False).max()
    .reset_index(level=1)
    .drop('China')
    .rename(index=name2map)
    .set_index('date_str',append=True)
)
china_df['existed'] = (china_df['confirmed'] - china_df['cured'] - china_df['dead'])


### Clean data
- linear interpolate
- fillna(0)
- +0.1, log10

In [5]:

clean_df = pd.DataFrame(
    index=pd.MultiIndex.from_product([geo_data.keys(),date_str]),
    columns=china_df.columns,
)

for g,df in china_df.reset_index().groupby('Name',sort=False):
    ndf = df.set_index('date_str').reindex(index=date_str).assign(Name=g).set_index('Name',append=True).interpolate().fillna(0)
    clean_df.loc[ndf.index.swaplevel(0,1)] = ndf.values

clean_df['color'] = plt.cm.Reds(Normalize(0, 5)(np.log10(clean_df['existed'].astype(int)+0.1).values)).tolist()



---

In [6]:
for n,d in tqdm(enumerate(date_str), total=date_str.shape[0]):
    
    if os.path.exists('frames/frame_{:02d}.png'.format(n)):
        continue
    
    ndf = clean_df.xs(d,level=1)
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(projection=ccrs.LambertConformal(central_latitude=90,central_longitude=105))
    
    ax.set_extent([80, 130, 13, 55])
    ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor='white', alpha=0.5)
    ax.add_feature(cfeature.OCEAN.with_scale('110m'))
    ax.add_feature(cfeature.COASTLINE.with_scale('110m'), zorder=1000)
    ax.add_feature(cfeature.BORDERS.with_scale('110m'), zorder=1000)
    
    for k in geo_data.keys():
        if k in ndf.index:
            gcolor = ndf.loc[k,'color']
        else:
            gcolor = 'white'
    
        ax.add_geometries(
            geo_data[k],
            crs=ccrs.PlateCarree(),
            facecolor=gcolor,
            lw=0.1,
            edgecolor='k',
            zorder=0
        )
    
    ax.set_title(d, fontsize=24)
    
    cax = fig.add_axes([0.825, 0.2, 0.02, 0.2])
    
    fig.colorbar(
        plt.cm.ScalarMappable(norm=Normalize(0, 5), cmap='Reds'), 
        cax=cax
    )
    
    cax.set_yticklabels(['$10^0$','$10^1$','$10^2$','$10^3$','$10^4$','$10^5$'], fontsize=12, ha='left',va='center')
    
    fig.savefig('frames/frame_{:02d}.png'.format(n), dpi=100, facecolor=None)
    
    plt.close(fig)
    
#     break

---

In [7]:
# fig = plt.figure(figsize=(8,8))
# ax = fig.add_subplot(projection=ccrs.LambertConformal(central_latitude=90,central_longitude=105))

# ax.set_extent([80, 130, 13, 55])
# ax.add_feature(cfeature.LAND.with_scale('50m'), facecolor='white', alpha=0.5)
# ax.add_feature(cfeature.OCEAN.with_scale('110m'))
# ax.add_feature(cfeature.COASTLINE.with_scale('110m'), zorder=1000)
# ax.add_feature(cfeature.BORDERS.with_scale('110m'), zorder=1000)

# for k in geo_data.keys():
#     if k in ndf.index:
#         c = ndf.loc[k,'color']
#     else:
#         c = 'white'
#     ax.add_geometries(
#         geo_data[k],
#         crs=ccrs.PlateCarree(),
#         facecolor=c,
#         lw=0.1,
#         edgecolor='k',
# #         hatch='//',
#         zorder=0
        
#     )
    
#     ax.set_title(d, fontsize=24)
    
#     cax = fig.add_axes([0.825, 0.2, 0.02, 0.2])
#     fig.colorbar(
#         plt.cm.ScalarMappable(norm=Normalize(0, 5), cmap='Reds'), 
#         cax=cax
#     )
    
#     cax.set_yticklabels(['$10^0$','$10^1$','$10^2$','$10^3$','$10^4$','$10^5$'], fontsize=12, ha='left',va='center')
    
#     plt.show()



In [8]:
# # https://gmt-china.org/
# with open('CN-border-La.dat') as src:
#     context = src.read()
#     blocks = [cnt for cnt in context.split('>') if len(cnt) > 0]
#     borders = [np.fromstring(block, dtype=float, sep=' ') for block in blocks]

# for line in borders:
#     ax.plot(
#         line[0::2], line[1::2],
#         '-', lw=1, color='k', 
#         transform=ccrs.Geodetic(),
#     )